In [1]:
import numpy as np
from openvino.inference_engine import IENetwork, IECore
from openvino.inference_engine import IEPlugin
import os
import cv2
import argparse

In [83]:
vid_man = './resources/manufacturing.mp4'

In [3]:
cpu_extension = '/opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_sse4.so'

In [76]:
mod_bin = './intel/person-detection-retail-0013/FP16/person-detection-retail-0013.bin'
mod_xml = './intel/person-detection-retail-0013/FP16/person-detection-retail-0013.xml'

In [77]:
plugin = IECore()
network = IENetwork(model=mod_xml, weights=mod_bin)

In [ ]:
#plugin.add_extension(cpu_extension, 'CPU')


In [6]:
exec_network = plugin.load_network(network,device_name='CPU')
        
input_blob = next(iter(network.inputs))
output_blob = next(iter(network.outputs))

In [7]:
net_input_shape=network.inputs[input_blob].shape

In [58]:
def exec_net(image):
        ### Start an asynchronous request 
    exec_network.start_async(request_id=0, inputs={input_blob: image})
    while True:
        status = exec_network.requests[0].wait(-1)
        if status == 0:
            break
        else:
            time.sleep(1)
    return

def wait(self):
    status = exec_network.requests[0].wait(-1)
    return status

     #Queue Parameters
        # For retail
        #queue.add_queue([620, 1, 915, 562])
        #queue.add_queue([1000, 1, 1264, 461])
        # For manufacturing
        #queue.add_queue([15, 180, 730, 780])
        #queue.add_queue([921, 144, 1424, 704])
        # For Transport 
        #queue.add_queue([50, 90, 838, 794])
        #queue.add_queue([852, 74, 1430, 841])


lenr = [[620, 1, 915, 562], [1000, 1, 1264, 461]]
lenm = [[15, 180, 730, 780],[921, 144, 1424, 704]]
lent = [[50, 90, 838, 794],[1000, 74, 1915, 841]]
       
net_input_shape[2]

320

In [67]:
# Get and open video capture
cap = cv2.VideoCapture(vid_man)
cap.open(vid_man)  
width = int(cap.get(3))
height = int(cap.get(4))
#Creating a video recorder
fourcc=cv2.VideoWriter_fourcc('M','J','P','G')
#fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('out_res.avi', fourcc, 10, (width,height))
fps = cap.get(cv2.CAP_PROP_FPS)
print('Frames per second is {}'.format(fps))
count =0
while cap.isOpened():
          
        # Read the next frame
        flag, frame = cap.read()
        #print(flag)
        if not flag or count ==200:
            break
        key_pressed = cv2.waitKey(60)
        #counter_frame += 1
        
        for i in range(len(lent)):
            x1,y1,x2,y2 = lent[i]
            p_frame1 = frame[y1:y2,x1:x2]
            t = np.random.randint(10000)
            count = count+1
            print('count',count)
            cv2.imwrite('./out_check_t/img'+str(count)+'.jpg',p_frame1)
        # Pre-process the frame
            p_frame = cv2.resize(p_frame1, (net_input_shape[3], net_input_shape[2]))
            p_frame = p_frame.transpose((2,0,1))
            p_frame = p_frame.reshape(1, *p_frame.shape)
          
            exec_network.start_async(request_id=0, inputs={input_blob: p_frame})
            while True:
                status = exec_network.requests[0].wait(-1)
                
                if status == 0:
                    break
                else:
                    time.sleep(1)
            status = exec_network.requests[0].wait(-1)
            if status ==0:
                #print('exec')
                result = exec_network.requests[0].outputs[output_blob]
                count_person(result,count)
            #print(result[0])
            
        #cap.release()
        #cv2.destroyAllWindows()

Frames per second is 29.97002997002997
count 1
Number of person in queue_1: 1
count 2
Number of person in queue_2: 1
Number of person in queue_2: 2
count 3
Number of person in queue_1: 1
count 4
Number of person in queue_2: 1
Number of person in queue_2: 2
count 5
Number of person in queue_1: 1
count 6
Number of person in queue_2: 1
Number of person in queue_2: 2
count 7
Number of person in queue_1: 1
count 8
Number of person in queue_2: 1
Number of person in queue_2: 2
count 9
Number of person in queue_1: 1
Number of person in queue_1: 2
count 10
Number of person in queue_2: 1
Number of person in queue_2: 2
count 11
Number of person in queue_1: 1
Number of person in queue_1: 2
count 12
Number of person in queue_2: 1
Number of person in queue_2: 2
count 13
Number of person in queue_1: 1
Number of person in queue_1: 2
count 14
Number of person in queue_2: 1
count 15
Number of person in queue_1: 1
Number of person in queue_1: 2
count 16
count 17
Number of person in queue_1: 1
Number of p

count 153
Number of person in queue_1: 1
count 154
Number of person in queue_2: 1
Number of person in queue_2: 2
count 155
Number of person in queue_1: 1
count 156
Number of person in queue_2: 1
Number of person in queue_2: 2
count 157
Number of person in queue_1: 1
Number of person in queue_1: 2
Number of person in queue_1: 3
count 158
Number of person in queue_2: 1
Number of person in queue_2: 2
count 159
Number of person in queue_1: 1
Number of person in queue_1: 2
count 160
Number of person in queue_2: 1
Number of person in queue_2: 2
count 161
Number of person in queue_1: 1
count 162
Number of person in queue_2: 1
Number of person in queue_2: 2
count 163
Number of person in queue_1: 1
count 164
Number of person in queue_2: 1
Number of person in queue_2: 2
Number of person in queue_2: 3
count 165
Number of person in queue_1: 1
count 166
Number of person in queue_2: 1
Number of person in queue_2: 2
Number of person in queue_2: 3
count 167
Number of person in queue_1: 1
count 168
Num

In [62]:
def count_person(result,count):
   d=0
   for i in range(200):
    if result[0][0][i][2]>0.5:
        #print('A person detected')
        d+=1
        if count%2 == 0:
            print('Number of person in queue_2: {}'.format(d)) 
        else:
            print('Number of person in queue_1: {}'.format(d))

In [53]:
np.random.randint(10)

3

In [26]:
import matplotlib.pyplot as plt
#plt.imshow(p_frame)

In [80]:
'.'+mod_bin.split('.')[1]+'.xml'

'./intel/person-detection-retail-0013/FP16/person-detection-retail-0013.xml'

In [33]:
result

array([[[[0.        , 1.        , 0.96114606, ..., 0.2084924 ,
          1.000035  , 0.9929421 ],
         [0.        , 1.        , 0.5607138 , ..., 0.48588908,
          0.03641473, 0.59447443],
         [0.        , 1.        , 0.39870456, ..., 0.08102133,
          0.32797396, 0.21776837],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)